# Visual Analytics Stock Market Project

## Initialization

In [ ]:
!pip install gspread oauth2client   #Install and import all required modules for updating and accessing google sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pprint
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from gspread_dataframe import set_with_dataframe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gdown      #Install module to download shared files from the google drive into python
import gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
#Download all the required files into the runtime
url = ""  #get URL for shared service acount client key setup to edit output google sheets
output = "client_key.json"
gdown.download(url,output, fuzzy = True)
url1 = "https://drive.google.com/file/d/1E-BrTXdMGDfoZ_EWjW9gqLhRcm8T9yuS/view?usp=sharing"
output1 = "nasdaq_screener.csv"
gdown.download(url1,output1, fuzzy = True)
url2 = "https://drive.google.com/file/d/1Hg5ZDKR3ba7Uo-owCepZNJruPgYU-1r3/view?usp=sharing"
output2 = "nyse_screener.csv"
gdown.download(url2,output2, fuzzy = True)
url3 = "https://drive.google.com/file/d/160b46wSwWGS3aVFRpqE1AXn9aF-BX4Km/view?usp=sharing"
output3 = "amex_screener.csv"
gdown.download(url3,output3, fuzzy = True)

NameError: ignored

In [ ]:
import pandas as pd     #read the imported csv files
#etf = pd.read_csv('etf_screener.csv')
nasdaq = pd.read_csv('nasdaq_screener.csv')
nyse = pd.read_csv('nyse_screener.csv')
amex = pd.read_csv('amex_screener.csv')

In [ ]:
#Removing all Stocks with extremely low volume trades
nasdaq.drop(nasdaq[nasdaq.Volume < 5000].index, inplace = True)
nyse.drop(nyse[nyse.Volume < 5000].index, inplace = True)
amex.drop(amex[amex.Volume < 5000].index, inplace = True)
#Removing all Stocks with market cap below 1000000000
nasdaq.drop(nasdaq[nasdaq['Market Cap'] < 10000000000].index, inplace = True)
nyse.drop(nyse[nyse['Market Cap'] < 10000000000].index, inplace = True)
amex.drop(amex[amex['Market Cap'] < 10000000000].index, inplace = True)
#Removing all Stocks with NaN
nasdaq.dropna(subset = ['Market Cap'], inplace = True)
nyse.dropna(subset = ['Market Cap'], inplace = True)
amex.dropna(subset = ['Market Cap'], inplace = True)

In [ ]:
#Remove columns that are no longer required
#etf.drop(etf.columns[2:],axis=1,inplace=True)
nasdaq.drop(nasdaq.columns[2:],axis=1,inplace=True)
nyse.drop(nyse.columns[2:],axis=1,inplace=True)
amex.drop(amex.columns[2:],axis=1,inplace=True)

In [ ]:
#Rename the Column headers
#etf.columns = ['symbol', 'name']
nasdaq.columns = ['symbol', 'name']
nyse.columns = ['symbol', 'name']
amex.columns = ['symbol', 'name']

In [ ]:
#Add a column that contains the stock exchange from which the stocks are
#etf['Stock Exchange'] = "ETF" 
nasdaq['Stock Exchange'] = "NASDAQ" 
nyse['Stock Exchange'] = "NYSE" 
amex['Stock Exchange'] = "AMEX" 
#Replace wrong symbols in the stock tickers
nasdaq['symbol'].replace("/", "-",regex = True, inplace = True)
nyse['symbol'].replace("/", "-",regex = True, inplace = True)
amex['symbol'].replace("/", "-",regex = True, inplace = True)
#Remove Tickers creating Problems
nasdaq = nasdaq[nasdaq["symbol"]!="GEN"]
nasdaq = nasdaq[nasdaq["symbol"]!="MBLY"]
nasdaq = nasdaq[nasdaq["symbol"]!="NWS"]
nyse = nyse[nyse["symbol"]!="BBDO"]
nyse = nyse[nyse["symbol"]!="BN"]
nyse = nyse[nyse["symbol"]!="HLN"]

In [ ]:
#Combine the different stock exchanges
market = pd.concat([nasdaq,nyse,amex])

In [ ]:
#Authorize the API to access and edit google sheets
scope = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file'
    ]
file_name = 'client_key.json'
creds = ServiceAccountCredentials.from_json_keyfile_name(file_name,scope)
client = gspread.authorize(creds)
gc = gspread.authorize(creds)
gauth = GoogleAuth()
drive = GoogleDrive(gauth)


In [ ]:
#Fetch all the sheets of the workbooks
gs = gc.open_by_key("11LBMHuBLuKapyIlemy2AL2HGoC-KgnD77lv82HCTr98") #Workbook 1 containging all exchange market stocks
gs2 = gc.open_by_key("1CDGWF8Fbhikbyt7vtWgmf6OFhbfH6IDLOBr00H-MtTE")  #workbook 2 containing all information on the stocks
wknasdaq = gs.worksheet('NASDAQ')
wknyse = gs.worksheet('NYSE')
wkamex = gs.worksheet('AMEX')
wkinfo = gs2.worksheet('Info')
wkrec = gs2.worksheet('Recommendations')
wkfin = gs2.worksheet('Financials') 
wkshare = gs2.worksheet('Shareholders') 
wksus = gs2.worksheet('Sustainability') 
wknews = gs2.worksheet('News') 
wkpt = gs2.worksheet('Price Target') 
wkfore = gs2.worksheet('Forecasts')  

## Data Collection Functions

In [ ]:
!pip install yfinance   #Install module to download stock information

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 12.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [ ]:
#import all required modules
import yfinance as yf
import datetime
from datetime import date, datetime, timezone
from dateutil.relativedelta import relativedelta

def History(stock):       #Function to get the past data of the stock prices and volume; Input is a Dataframe containing the stock tickers
    hist = pd.DataFrame(columns = ['Symbol', 'Stock Exchange', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']) #Dataframe to contain all stock price data
    for i in range(len(stock)):   #For loop to iterate through the stock tickers in the input dataframe
        tick = str(stock.iloc[i,0])   #Get the stock ticker from the input
        obj = yf.Ticker(tick)         #Determine the yahoo finance Ticker object
        y = pd.DataFrame(obj.history(period="1mo").iloc[-1])  #Determine the last day the stock was traded
        y = y.transpose()
        y.reset_index(inplace=True)
        y = y.rename(columns= {y.columns[0]: "Date"})
        y['Date'] = pd.to_datetime(y['Date']).dt.date       #Change the date to appropriate format
        today = y.iloc[0]['Date'] - relativedelta(days = 1) #Determine today(yesterday), day before the last stock price update
        D1 = today - relativedelta(years = 2)               #Determine the different time periods, for which the data will be collected differently
        D1 = datetime.strptime((str(D1)[:-2] + "02"), '%Y-%m-%d').date()
        D2 = today - relativedelta(years = 1)
        D2 = datetime.strptime((str(D2)[:-2] + "02"), '%Y-%m-%d').date()
        D3 = today - relativedelta(months = 2)
        D4 = today - relativedelta(days = 7)
        D5 = today - relativedelta(days = 5)
        a = pd.DataFrame(obj.history(period="max").iloc[0]) #Determine the earliest stock data available
        a = a.transpose()
        a.reset_index(inplace=True)
        a = a.rename(columns= {a.columns[0]: "Date"})
        a['Date'] = pd.to_datetime(a['Date']).dt.date       #Convert the date to the appropriate format
        b = a.iloc[0]['Date']
        print(tick, today, b, D1, D2, D3, D4, D5)
        if b < D1 and ((D1.year - b.year) *12 + D1.month -b.month) > 3 :    #Check that the stock has data at least 3 months before the date D1
            df1 = pd.DataFrame.from_dict(obj.history(start= "2000-01-01", end = D1, interval = "3mo"))  #Get the history divided in different periods and intervals
            df2 = pd.DataFrame.from_dict(obj.history(start= D1, end = D2, interval = "1mo"))
            df3 = pd.DataFrame.from_dict(obj.history(start= D2, end = D3, interval = "1wk"))
            df4 = pd.DataFrame.from_dict(obj.history(start= D3, end = D4, interval = "1d"))
            df5 = pd.DataFrame.from_dict(obj.history(start= D4, end = D5, interval = "1h"))
            df6 = pd.DataFrame.from_dict(obj.history(start = D5, interval = "15m"))
            df = pd.concat([df1,df2,df3,df4,df5,df6])     #Combine the dataframes
            df.drop(df.columns[5:],axis=1,inplace=True)   #Drop unnecessary columns
            df.reset_index(inplace=True)                  #reset the index, as it is the date
            df = df.rename(columns= {df.columns[0]: "Date"})  #rename the old index to Date
            df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert(None)   #Ensure values in the date column are in the right format
            df.insert(0, 'Symbol', tick)    #Add a column for the stock ticker to the df
            hist = pd.concat([hist, df])    #Add the dataframe to the complete collection 
        elif b < D2 and ((D2.year - b.year) *12 + D2.month - b.month) > 1:  #Check that the b is at least 1 month befoer D2
            df2 = pd.DataFrame.from_dict(obj.history(start= "2000-01-01", end = D2, interval = "1mo"))  #Repeat above
            df3 = pd.DataFrame.from_dict(obj.history(start= D2, end = D3, interval = "1wk"))
            df4 = pd.DataFrame.from_dict(obj.history(start= D3, end = D4, interval = "1d"))
            df5 = pd.DataFrame.from_dict(obj.history(start= D4, end = D5, interval = "1h"))
            df6 = pd.DataFrame.from_dict(obj.history(start= D5, interval = "15m"))
            df = pd.concat([df2,df3,df4,df5,df6])
            df.drop(df.columns[5:],axis=1,inplace=True)
            df.reset_index(inplace=True)
            df = df.rename(columns= {df.columns[0]: "Date"})
            df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert(None)
            df.insert(0, 'Symbol', tick)
            hist = pd.concat([hist, df])
        elif b > D3 and ((D3 - b).days) > 14:   #Check that b is at least 14 days before D3
            df3 = pd.DataFrame.from_dict(obj.history(start= "2000-01-01", end = D3, interval = "1wk"))
            df4 = pd.DataFrame.from_dict(obj.history(start= D3, end = D4, interval = "1d"))
            df5 = pd.DataFrame.from_dict(obj.history(start= D4, end = D5, interval = "1h"))
            df6 = pd.DataFrame.from_dict(obj.history(start= D5, interval = "15m"))
            df = pd.concat([df3,df4,df5,df6])
            df.drop(df.columns[5:],axis=1,inplace=True)
            df.reset_index(inplace=True)
            df = df.rename(columns= {df.columns[0]: "Date"})
            df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert(None)
            df.insert(0, 'Symbol', tick)
            hist = pd.concat([hist, df])
        elif b < D4 and ((D4 - b).days) > 5:    #Check that b is at least 5 days before D5
            df4 = pd.DataFrame.from_dict(obj.history(start= "2000-01-01", end = D4, interval = "1d"))
            df5 = pd.DataFrame.from_dict(obj.history(start= D4, end = D5, interval = "1h"))
            df6 = pd.DataFrame.from_dict(obj.history(start= D5, interval = "15m"))
            df = pd.concat([df4,df5,df6])
            df.drop(df.columns[5:],axis=1,inplace=True)
            df.reset_index(inplace=True)
            df = df.rename(columns= {df.columns[0]: "Date"})
            df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert(None)
            df.insert(0, 'Symbol', tick)
            hist = pd.concat([hist, df])
        elif b < D5:    #Check that b is smaller than D5
            df5 = pd.DataFrame.from_dict(obj.history(start= "2000-01-01", end = D5, interval = "1h"))
            df6 = pd.DataFrame.from_dict(obj.history(start= D5, interval = "15m"))
            df = pd.concat([df5,df6])
            df.drop(df.columns[5:],axis=1,inplace=True)
            df.reset_index(inplace=True)
            df = df.rename(columns= {df.columns[0]: "Date"})
            df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert(None)
            df.insert(0, 'Symbol', tick)
            hist = pd.concat([hist, df])
        else:           #If stock was only listed recently
            df6 = pd.DataFrame.from_dict(obj.history(start= D5, interval = "15m"))
            df = df6
            df.drop(df.columns[5:],axis=1,inplace=True)
            df.reset_index(inplace=True)
            df = df.rename(columns= {df.columns[0]: "Date"})
            df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert(None)
            df.insert(0, 'Symbol', tick)
            hist = pd.concat([hist, df])
    hist['Stock Exchange'] = stock['Stock Exchange'].iloc[0]  #Add a column denoting the stock exchange the stock is from
    return hist   #return the full stock price history of all input tickers
        
def Info(stock):    #Function to get information about the stocks from a dataframe of stock tickers
    info = pd.DataFrame(columns = ['Symbol', 'Name', 'Summary', 'Country', 'Sector', 'Industry', 'Employees', 
                                   'Website', 'Logo URL', 'Ask', 'Ask Size', 'Bid', 'Bid Size', 'Profit Margins', 
                                   'Revenue Growth', ' Return on Assets', 'Debt to Equity', '52 Week Change', 
                                   '52 Week High', '52 Week Low', 'Trailing Peg', 'Last Dividend Value',
                                   'Last Dividend Date', 'Five Year Average Dividend', 'Recommendation Key', 
                                   'Recommendation Mean'])  #DF to store information on all stocks

    for i in range(len(stock)):
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        df = (pd.DataFrame.from_dict(obj.info, orient = 'index')).T   #Get the info as a dataframe with the right orientation
        print(tick, df.shape)
        try:    #Ensure that the df actually contains the required columns, and return " " if not, repeat for all values
          a = df._get_value(0, 'longName')
        except KeyError:
          try:
            a = df._get_value(0, 'shortName')
          except KeyError:
            a = stock.iloc[i,1]
          else:
            a = df._get_value(0, 'shortName')
        else:
          a = df._get_value(0, 'longName') 
        if a != "" and df.shape[1] < 100:   #If the shape of the df is not above 100 get the name from the input
          a = stock.iloc[i,1]
        try:
          b = df._get_value(0, 'longBusinessSummary')
        except KeyError:
          b = " "
        try:
          c = df._get_value(0, 'country')
        except KeyError:
          c = " "
        try:
          d = df._get_value(0, 'sector')
        except KeyError:
          d = " "
        try:
          e = df._get_value(0, 'industry')
        except KeyError:
          e = " "
        try:
          f = df._get_value(0, 'fullTimeEmployees')
        except KeyError:
          f = " "
        try:
          g = df._get_value(0,'website')
        except KeyError:
          g = " "
        try:
          h = df._get_value(0, 'logo_url')
        except KeyError:
          h = " "
        try:
          j = df._get_value(0, 'ask')
        except KeyError:
          j = " "
        try:
          k = df._get_value(0, 'askSize')
        except KeyError:
          k = " "
        try:
          l = df._get_value(0, 'bid')
        except KeyError:
          l = " "
        try:
          m = df._get_value(0, 'bidSize')
        except KeyError:
          m = " "
        try:
          n = df._get_value(0, 'profitMargins')
        except KeyError:
          n = " "
        try:
          o = df._get_value(0, 'revenueGrowth')
        except KeyError:
          o = " "
        try:
          p = df._get_value(0, 'returnOnAssets')
        except KeyError:
          p = " "
        try:
          q = df._get_value(0, 'debtToEquity')
        except KeyError:
          q = " "
        try:
          r = df._get_value(0, '52WeekChange')
        except KeyError:
          r = " "
        try:
          s = df._get_value(0, 'fiftyTwoWeekHigh')
        except KeyError:
          s = " "
        try:
          t = df._get_value(0, 'fiftyTwoWeekLow')
        except KeyError:
          t = " "
        try:
          u = df._get_value(0, 'trailingPegRatio')
        except KeyError:
          u = " "
        try:
          v = df._get_value(0, 'lastDividendValue')
        except KeyError:
          v = " "
        try:
          w = df._get_value(0, 'lastDividendDate')
        except KeyError:
          w = " "
        try:
          x = df._get_value(0, 'fiveYearAvgDividendYield')
        except KeyError:
          x = " "
        try:
          y = df._get_value(0, 'recommendationKey')
        except KeyError:
          y = " "
        try:
          z = df._get_value(0, 'recommendationMean')
        except KeyError:
          z = " "
        temp = pd.DataFrame([[tick, a, b, c, d, e, f, g, h, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z]] ,columns = ['Symbol', 'Name', 'Summary', 'Country', 'Sector', 'Industry', 'Employees', 
                                       'Website', 'Logo URL', 'Ask', 'Ask Size', 'Bid', 'Bid Size', 'Profit Margins', 
                                       'Revenue Growth', ' Return on Assets', 'Debt to Equity', '52 Week Change', 
                                       '52 Week High', '52 Week Low', 'Trailing Peg', 'Last Dividend Value',
                                       'Last Dividend Date', 'Five Year Average Dividend', 'Recommendation Key', 
                                       'Recommendation Mean'])  #store all the information on one stock temporarly
        info = pd.concat([info,temp])     #Add the stock information to the table with all stock information
    return info   #return all stock information

def Recommendations(stock):   #Function to get stock recommendation from input dataframe with tickers
    rec = pd.DataFrame(columns = ['Symbol', 'Date', 'Firm', 'Grade'])     #create the output dataframe
    today =(date.today())   #get the data today
    D1 = today - relativedelta(months = 6)    #get the date 6 months ago, so that only recommendations over the last 6 months are taken into account
    for i in range(len(stock)):     #for loop to go through the stock tickers
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        df = pd.DataFrame(obj.recommendations)    #get a dataframe of the stock recommendations
        df.reset_index(inplace=True)      #reset the index containing the date
        df = df.rename(columns= {df.columns[0]: "Date"})    #rename the column to date
        df['Date'] = pd.to_datetime(df['Date'])     #convert the date column to datetime type
        df['Date'] = df['Date'].dt.date             #convert date column to date
        df.insert(0, 'Symbol', tick)                #insert the stock ticker as the first column of the dataframe
        df = df.rename(columns= {"To Grade": "Grade"})    #rename a column
        df.drop(df.columns[4:],axis=1,inplace=True)       #remove unnecessary columns
        df.drop(df[df.Date < D1].index, inplace = True)   #drop dates older than 6 months
        df.reset_index(inplace = True, drop = True)       #reset the index
        rec = pd.concat([rec, df])                  #append the dataframe to the output dataframe
    return rec    #return the recommendations of all stocks
        
def Financials(stock):    #function to get financial information about the companies
    fin = pd.DataFrame(columns = ['Symbol', 'Date', 'Operating Cashflow', 'Free Cashflow', 'Current Liabilities', 
                                  'Working Capital', 'Invested Capital', 'Total Revenue', 'Cost of Revenue', 
                                  'Gross Profit', 'EBIT', "Operating Cashflow Ratio"])  #create output dataframe
    for i in range(len(stock)):   #for loop to go through the stock tickers
        cash = pd.DataFrame(columns = ['Date', 'Operating Cashflow', 'Free Cashflow'])    #cashflow dataframe format
        bal = pd.DataFrame(columns = ['Date', 'Current Liabilities', 'Working Capital', 'Invested Capital'])    #balance sheet dataframe format
        inc = pd.DataFrame(columns = ['Date', 'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'EBIT' ])   #income statement dataframe format
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        df1 = pd.DataFrame(obj.cashflow)  #get dataframe of cashflow
        df1t = df1.transpose()    #transpose to search by column
        df1t = df1t.reset_index() #reset the index containing the date
        df1t = df1t.rename(columns={df1t.columns[0]:'Date'})  #rename the column to Date
        df1t['Date'] = pd.to_datetime(df1t['Date'])     #convert to right date format
        if df1t.empty:    #if the company financials can not be found
          cash = pd.DataFrame([["", "", ""]], columns = ['Date', 'Operating Cashflow', 'Free Cashflow'])    #cash is empty
        else:   
          cash['Date'] = df1t['Date'].dt.date     
          cash['Operating Cashflow'] = df1t.get('Operating Cash Flow')
          cash['Free Cashflow'] = df1t.get('Free Cash Flow')    #equate the cashflow dataframe to the cashflow data
        df2 = pd.DataFrame(obj.balance_sheet)   #repeat the same process for balance sheet and income statement data
        df2t = df2.transpose()    
        df2t.reset_index(inplace=True)
        df2t = df2t.rename(columns={df2t.columns[0]:'Date'})
        df2t['Date'] = pd.to_datetime(df2t['Date'])
        if df2t.empty:
          bal = pd.DataFrame([["", "", "", ""]], columns = ['Date', 'Current Liabilities', 'Working Capital', 'Invested Capital'])
        else:
          bal['Date'] = df2t['Date'].dt.date
          bal['Current Liabilities'] = df2t.get('Current Liabilities')
          bal['Working Capital'] = df2t.get('Working Capital')
          bal['Invested Capital'] = df2t.get('Invested Capital')
        df3 = pd.DataFrame(obj.income_stmt)
        df3t = df3.transpose()
        df3t.reset_index(inplace=True)
        df3t = df3t.rename(columns={df3t.columns[0]:'Date'})
        df3t['Date'] = pd.to_datetime(df3t['Date'])
        if df3t.empty:
          df3t = pd.DataFrame([["", "", "", "", ""]], columns = ['Date', 'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'EBIT' ])
        else:
          inc['Date'] = df3t['Date'].dt.date
          inc['Total Revenue'] = df3t.get('Total Revenue')
          inc['Cost of Revenue'] = df3t.get('Cost Of Revenue')
          inc['Gross Profit'] = df3t.get('Gross Profit')
          inc['EBIT'] = df3t.get('EBIT')
        df = pd.merge(pd.merge(cash,bal, on = 'Date', how = "outer"), inc, on = 'Date', how = "outer")    #merge the cashflow, balance sheet and income statement into 1 dataframe
        df.insert(0, 'Symbol', tick)    #insert the stock tickers
        try:  #try whether the calculation of the Operating cashflow ratio is possible
          df['Operating Cashflow Ratio'] = df['Operating Cashflow'] / df['Current Liabilities']
        except TypeError: #if a TypeError is returned the Operating Cashflow Ratio cannot be calculated and remains empty
          df['Operating Cashflow Ratio'] = ""
        fin = pd.concat([fin,df]) #append the dataframe to the output dataframe
    return fin    #return the output dataframe

def Shareholders(stock):    #Get information on who are the major shareholders
    sh = pd.DataFrame(columns = ['Symbol', 'Percentage', 'Shareholder'])    #format output dataframe
    for i in range(len(stock)): #for loop to iterate through tickers
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        print(tick, ((i/len(stock)*100)))
        df = pd.DataFrame(obj.major_holders)  #get data on the major shareholders of the company as a dataframe
        if df.empty:  #if the dataframe is empty, no information is available
          df = pd.DataFrame([['Not Available', 'Not Available']], columns = ['Percentage', 'Shareholder'])
        else: #if information is available
          df = df.rename({df.columns[0]: 'Percentage', df.columns[1]: 'Shareholder'}, axis = 'columns') #rename the columns
        df.insert(0, 'Symbol', tick)  #insert the stock ticker in a new first column
        sh = pd.concat([sh,df])   #append to the output dataframe
    return sh   #return the output dataframe
        
def Sustainability(stock):  #Function to get data on sustainability
    sus = pd.DataFrame(columns = ['Symbol', 'Total ESG Risk Score', 'Social Risk Score', 'Governance Risk Score', 
                                  'Environmental Risk Score', 'Percentile', 'Highest Controversy'])   #format output dataframe
    for i in range(len(stock)):   #for loop to iterate through tickers
        temp = pd.DataFrame(columns = ['Symbol', 'Total ESG Risk Score', 'Social Risk Score', 'Governance Risk Score', 
                                       'Environmental Risk Score','Percentile', 'Highest Controversy']) #format temporary dataframe
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        print(tick, ((i/len(stock)*100)))
        df = pd.DataFrame(obj.sustainability)   #input the sustainability data into a dataframe
        if df.empty:  #if the dataframe is empty all values except the symbol are none, as no data is available
          temp = pd.DataFrame([[tick, "None", "None", "None", "None", "None", "None"]], columns =['Symbol', 'Total ESG Risk Score', 'Social Risk Score', 'Governance Risk Score', 
                             'Environmental Risk Score','Percentile', 'Highest Controversy'])
        else:   #otherwise get the values from the dataframe and input them into the temporary dataframe
          a = df._get_value('totalEsg', 'Value')
          b = df._get_value('socialScore', 'Value')
          c = df._get_value('governanceScore', 'Value')
          d = df._get_value('environmentScore', 'Value')
          e = df._get_value('percentile', 'Value')
          f = df._get_value('highestControversy', 'Value')
          temp = pd.DataFrame([[tick, a, b, c, d, e, f]], columns =['Symbol', 'Total ESG Risk Score', 'Social Risk Score', 'Governance Risk Score', 
                             'Environmental Risk Score','Percentile', 'Highest Controversy'])
        sus = pd.concat([sus,temp])   #append the temporary dataframe to the output dataframe
    return sus    #return the output dataframe
        
def News(stock):  #Function to get relevant news around the companies
    news = pd.DataFrame(columns = ['Symbol', 'Title', 'Publisher', 'Link'])   #format output dataframe
    for i in range(len(stock)):   #for loop to iterate through tickers
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        print(tick, ((i/len(stock)*100)))
        df = pd.DataFrame(obj.news)   #input news data into a dataframe
        df.drop(df.columns[4:], axis = 1, inplace = True)   #drop irrelevant columns
        df.drop(df.columns[0], axis = 1, inplace = True)    #drop irrelevant column
        df = df.rename(columns= {"title": "Title","publisher":"Publisher","link":"Link"}) #rename columns to line up with output dataframe
        df.insert(0, 'Symbol', tick)  #insert stock ticker column
        news = pd.concat([news,df])   #append dataframe to output dataframe
    return news   #return output dataframe

def PriceTarget(stock):   #Function to get analyst stock price estimates
    pt = pd.DataFrame(columns = ['Symbol', 'Target Low Price', 'Current Price', 'Target Mean Price', 'Target High Price', 
                                 'Number of Analyst Opinions'])   #format output dataframe
    for i in range(len(stock)):   #for loop to iterate through tickers
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        print(obj)
        df = pd.DataFrame(obj.analyst_price_target)   #input analyst price targets into dataframe
        if df.empty: #if the dataframe is empty all values except the symbol are empty, as no data is available
          temp = pd.DataFrame([[tick, " ", " ", " ", " ", " "]], columns = ['Symbol', 'Target Low Price', 'Current Price', 'Target Mean Price', 
                                       'Target High Price', 'Number of Analyst Opinions'])
        else: #otherwise get the values from the dataframe and input them into the temporary dataframe
          a = df._get_value('targetLowPrice', df.columns[0])
          b = df._get_value('currentPrice', df.columns[0])
          c = df._get_value('targetMeanPrice', df.columns[0])
          d = df._get_value('targetHighPrice', df.columns[0])
          e = df._get_value('numberOfAnalystOpinions', df.columns[0])
          temp = pd.DataFrame([[tick, a, b, c, d, e]], columns = ['Symbol', 'Target Low Price', 'Current Price', 'Target Mean Price', 
                                       'Target High Price', 'Number of Analyst Opinions'] )
        pt = pd.concat([pt,temp])   #append dataframe to output dataframe
    return pt   #return output dataframe
        
def Forecasts(stock):   #Function to get financial forecasts
    fore = pd.DataFrame(columns = ['Symbol', 'End Date', 'Average Earnings Forecast', 'Low Earnings Forecast', 
                                  'High Earnings Forecast', 'Earnings per Share previous Year', "Earnings Growth",  
                                  'Number of Analysts Earnings', 'Average Revenue Forecast', 'Low Revenue Forecast',
                                  'High Revenue Forecast', 'Year Ago Sales','Sales Growth', 'Number of Analysts Revenue'])    #format output dataframe
    for i in range(len(stock)):   #for loop to iterate through tickers
        tick = str(stock.iloc[i,0])
        obj = yf.Ticker(tick)
        a = obj.earnings_forecasts.to_dict()      #get the earnings forecast data as a dictionary
        df1 = pd.DataFrame.from_dict(a, orient = "columns")   #turn the dictionary into a dataframe
        if df1.empty: #if the dataframe is empty, pass the ticker to the f2 function and try again
          df1 = f2(tick)
        else: #otherwise use the dataframe
          df1 = df1

        df1 = df1.rename(columns= {"avg":"Average Earnings Forecast","low":"Low Earnings Forecast",
                                   "high":"High Earnings Forecast","yearAgoEps":"Earnings per Share previous Year",
                                   "growth":"Earnings Growth", "numberOfAnalysts":"Number of Analysts Earnings",
                                   "endDate":"End Date"})   #rename the columns in the dataframe
        print(tick, ((i/len(stock)*100)), type(df1), df1.shape)
        df1.drop(df1.columns[-2], axis = 1, inplace = True)   #drop irrelevant columns
        df1.dropna(inplace=True)    #drop rows with no data
        df2 = pd.DataFrame.from_dict(obj.revenue_forecasts, orient = "columns")   #get the revenue forecasts as a dataframe
        if df2.empty: #if the dataframe is empty, pass the ticker to the f3 function and try again
          df2 = f3(tick)
        else: #otherwise use the dataframe
          df2 = df2

        df2 = df2.rename(columns= {"avg":"Average Revenue Forecast","low":"Low Revenue Forecast",
                                   "high":"High Revenue Forecast","yearAgoRevenue":"Year Ago Sales",
                                   "growth":"Sales Growth", "numberOfAnalysts":"Number of Analysts Revenue",
                                   "endDate":"End Date"})   #rename the columns
        df2.drop(df2.columns[-2], axis = 1, inplace = True)   #drop irrelevant columns
        df2.dropna(inplace=True)    #drop rows with no data
        df = df1.merge(df2, on = "End Date")    #merge the two forecasts on the End Date column
        df.dropna(inplace=True)   #drop empty rows
        df.insert(0, 'Symbol', tick)  #insert column with the stock ticker
        fore = pd.concat([fore,df])   #append dataframe to output dataframe
    return fore   #return output dataframe

def f2(tick):   #function to try to get earnings forecasts again, input is the stock ticker
        obj = yf.Ticker(tick)
        a = obj.earnings_forecasts.T
        df1 = pd.DataFrame.from_dict(a, orient = "columns").T
        if df1.empty:   #if the dataframe is still empty return dataframe with empty values
          df1 = pd.DataFrame([[" ", " ", " ", " ", 
                               " ", " "," "]], 
                             columns= ["avg","low", "high","yearAgoEps", "growth", "numberOfAnalysts",
                             "endDate"])
        return df1      #otherwise return the dataframe
        
def f3(tick):   #function to try to get revenue forecasts again, input is the stock ticker
        obj = yf.Ticker(tick)
        a = obj.revenue_forecasts.T
        df1 = pd.DataFrame.from_dict(a, orient = "columns").T
        if df1.empty:   #if the dataframe is still empty return dataframe with empty values
          df1 = pd.DataFrame([[" ", " ", " ", " ", " ", " "," "]], 
                             columns= ["avg","low", "high","yearAgoEps", "growth", "numberOfAnalysts",
                             "endDate"])
        return df1      #otherwise return the dataframe
        

## Testing

In [ ]:
stock = nasdaq.sample(n=2, random_state = 1)
print(stock)

     symbol                                  name Stock Exchange
1642   FPAY         FlexShopper Inc. Common Stock         NASDAQ
3141   OMGA  Omega Therapeutics Inc. Common Stock         NASDAQ


In [ ]:
h = History(stock)
print(h)

    Symbol Stock Exchange        Date  Open  High   Low  Close Volume
0     FPAY         NASDAQ  2010-04-01  5.40  5.40  5.00   5.00    230
1     FPAY         NASDAQ  2010-07-01  5.00  7.50  5.00   5.00    540
2     FPAY         NASDAQ  2010-10-01  5.00  5.00  5.00   5.00      0
3     FPAY         NASDAQ  2011-01-01  5.00  5.00  2.00   3.50   2280
4     FPAY         NASDAQ  2011-04-01  3.50  3.50  1.50   3.00   5620
..     ...            ...         ...   ...   ...   ...    ...    ...
117   OMGA         NASDAQ  2022-12-21  5.31  5.31  5.31   5.31    405
118   OMGA         NASDAQ  2022-12-21  5.29  5.30  5.29   5.30   1581
119   OMGA         NASDAQ  2022-12-21  5.17  5.18  5.17   5.18    461
120   OMGA         NASDAQ  2022-12-21  5.23  5.25  5.07   5.07   3233
121   OMGA         NASDAQ  2022-12-21  5.07  5.07  5.07   5.07      0

[288 rows x 8 columns]


In [ ]:
i = Info(stock)
print(i)

  Symbol                      Name  \
0   FPAY         FlexShopper, Inc.   
0   OMGA  Omega Therapeutics, Inc.   

                                             Summary        Country  \
0  FlexShopper, Inc., a financial technology comp...  United States   
0  Omega Therapeutics, Inc. operates as a clinica...  United States   

        Sector                   Industry Employees  \
0  Industrials  Rental & Leasing Services       137   
0   Healthcare              Biotechnology        79   

                         Website  \
0    https://www.flexshopper.com   
0  https://omegatherapeutics.com   

                                          Logo URL   Ask  ... Debt to Equity  \
0        https://logo.clearbit.com/flexshopper.com  1.25  ...        394.301   
0  https://logo.clearbit.com/omegatherapeutics.com  0.00  ...         17.126   

   52 Week Change 52 Week High  52 Week Low Trailing Peg  Last Dividend Value  \
0       -0.604348         3.17        0.830         None                 N

In [ ]:
r = Recommendations(stock)
print(r)

  Symbol        Date                 Firm Grade
0   FPAY  2022-08-12  HC Wainwright & Co.   Buy
1   FPAY  2022-11-14  HC Wainwright & Co.   Buy
0   OMGA  2022-09-23      Chardan Capital   Buy
1   OMGA  2022-12-08  HC Wainwright & Co.   Buy


In [ ]:
f = Financials(stock)
print(f)

        Date Operating Cash Flow  \
0 2021-12-31         -11256031.0   
1 2020-12-31          -5207547.0   
2 2019-12-31           -469461.0   
3 2018-12-31         -18160258.0   

  Cash Flow From Continuing Operating Activities  \
0                                    -11256031.0   
1                                     -5207547.0   
2                                      -469461.0   
3                                    -18160258.0   

  Net Income From Continuing Operations Operating Gains Losses  \
0                             3272774.0             -1931825.0   
1                             -339896.0                    NaN   
2                              577415.0                    NaN   
3                            -9461262.0               126622.0   

  Depreciation Amortization Depletion Depreciation And Amortization  \
0                          76487834.0                    76487834.0   
1                          63308210.0                    63308210.0   
2             

In [ ]:
s = Shareholders(stock)
print(s)

  Symbol Percentage                            Shareholder
0   FPAY     34.42%        % of Shares Held by All Insider
1   FPAY     22.43%       % of Shares Held by Institutions
2   FPAY     34.21%        % of Float Held by Institutions
3   FPAY         26  Number of Institutions Holding Shares
0   OMGA      3.69%        % of Shares Held by All Insider
1   OMGA     88.78%       % of Shares Held by Institutions
2   OMGA     92.18%        % of Float Held by Institutions
3   OMGA         68  Number of Institutions Holding Shares


In [ ]:
sus = Sustainability(stock)
print(sus)

  Symbol Total ESG Risk Score Social Risk Score Governance Risk Score  \
0   FPAY                 None              None                  None   

  Environmental Risk Score Percentile Highest Controversy  
0                     None       None                None  
  Symbol Total ESG Risk Score Social Risk Score Governance Risk Score  \
0   OMGA                 None              None                  None   

  Environmental Risk Score Percentile Highest Controversy  
0                     None       None                None  
  Symbol Total ESG Risk Score Social Risk Score Governance Risk Score  \
0   FPAY                 None              None                  None   
0   OMGA                 None              None                  None   

  Environmental Risk Score Percentile Highest Controversy  
0                     None       None                None  
0                     None       None                None  


In [ ]:
n = News(stock)
print(n)

  Symbol                                              Title        Publisher  \
0   FPAY  FlexShopper, Inc. Announces Purchase of Revolu...    GlobeNewswire   
1   FPAY  FlexShopper Third Quarter 2022 Earnings: Misse...  Simply Wall St.   
2   FPAY  FlexShopper Inc. (FPAY) Reports Q3 Loss, Misse...            Zacks   
3   FPAY  FlexShopper, Inc. Reports Third Quarter 2022 F...    GlobeNewswire   
4   FPAY  FlexShopper, Inc. Schedules 2022 Third Quarter...    GlobeNewswire   
5   FPAY            11 Best Nano-Cap Stocks To Buy For 2022   Insider Monkey   
6   FPAY  FlexShopper, Inc. Announces Exclusive Relation...    GlobeNewswire   
7   FPAY  FlexShopper Inc. (FPAY) Flat As Market Gains: ...            Zacks   
0   OMGA  Omega Therapeutics Could Offer 'Intriguing' Li...         Benzinga   
1   OMGA  Wall Street Analysts Think Omega Therapeutics,...            Zacks   
2   OMGA  Omega Therapeutics to Participate in the Piper...      PR Newswire   
3   OMGA  Omega Therapeutics Named One o

In [ ]:
pt = PriceTarget(stock)
print(pt)

  Symbol  Target Low Price  Current Price  Target Mean Price  \
0   FPAY               2.0           0.91               3.67   
0   OMGA               4.0           5.07              15.00   

   Target High Price  Number of Analyst Opinions  
0                6.0                         3.0  
0               30.0                         6.0  


In [ ]:
fore = Forecast(stock)
print(fore)

  Symbol    End Date  Average Earnings Forecast  Low Earnings Forecast  \
2   FPAY  2022-12-31                      -0.11                  -0.11   
3   FPAY  2022-12-31                      -0.11                  -0.11   
4   FPAY  2023-03-31                      -0.04                  -0.11   
5   FPAY  2023-12-31                      -0.05                  -0.26   
0   OMGA  2022-12-31                      -0.63                  -0.69   
1   OMGA  2022-12-31                      -0.63                  -0.69   
2   OMGA  2022-12-31                      -2.13                  -2.19   
3   OMGA  2022-12-31                      -2.13                  -2.19   
4   OMGA  2023-03-31                      -0.64                  -0.80   
5   OMGA  2023-12-31                      -2.71                  -3.00   

   High Earnings Forecast  Earnings per Share previous Year  Earnings Growth  \
2                   -0.11                              0.04           -3.750   
3                   -0.11

## Output

In [ ]:
n = History(nasdaq)     #Get stock history for nasdaq

AAPL 2023-01-16 1980-12-12 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ABMD 2022-12-22 1987-07-29 2020-12-02 2021-12-02 2022-10-22 2022-12-15 2022-12-17
ABNB 2023-01-16 2020-12-10 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ACGL 2023-01-16 1995-09-14 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ADBE 2023-01-16 1986-08-13 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ADI 2023-01-16 1980-03-17 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ADP 2023-01-16 1980-03-17 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ADSK 2023-01-16 1985-06-28 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
AEP 2023-01-16 1962-01-02 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
AKAM 2023-01-16 1999-10-29 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ALGN 2023-01-16 2001-01-30 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ALNY 2023-01-16 2004-06-01 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
AMAT 2023-01-16 198

In [ ]:
wknasdaq.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wknasdaq, dataframe=n, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
ny = History(nyse)    #Get stock history for nyse stocks

A 2023-01-16 1999-11-18 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ABB 2023-01-16 2001-04-06 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ABBV 2023-01-16 2013-01-02 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ABC 2023-01-16 1995-04-04 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ABEV 2023-01-16 1997-03-05 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ABT 2023-01-16 1980-03-17 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ACI 2023-01-16 2020-06-26 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ACM 2023-01-16 2007-05-10 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ACN 2023-01-16 2001-07-19 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
ADM 2023-01-16 1980-03-17 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
AEE 2023-01-16 1998-01-02 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
AEM 2023-01-16 1973-02-21 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
AER 2023-01-16 2006-11-21 20

In [ ]:
wknyse.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wknyse, dataframe=ny, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
a = History(amex)     #Get stock history for amex stocks

CQP 2023-01-16 2007-03-21 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
IMO 2023-01-16 1980-03-17 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11
LNG 2023-01-16 1994-04-04 2021-01-02 2022-01-02 2022-11-16 2023-01-09 2023-01-11


In [ ]:
wkamex.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkamex, dataframe=a, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
info = Info(market)     #Get all information data for all exchanges

AAPL (1, 154)
ABMD (1, 155)
ABNB (1, 154)
ACGL (1, 154)
ADBE (1, 154)
ADI (1, 154)
ADP (1, 154)
ADSK (1, 154)
AEP (1, 154)
AKAM (1, 155)
ALGN (1, 156)
ALNY (1, 155)
AMAT (1, 155)
AMD (1, 154)
AMGN (1, 155)
AMZN (1, 154)
ANSS (1, 155)
APA (1, 154)
ARGX (1, 152)
ASML (1, 153)
ATVI (1, 154)
AVGO (1, 154)
AXON (1, 153)
AZN (1, 154)
AZPN (1, 155)
BGNE (1, 154)
BIDU (1, 155)
BIIB (1, 154)
BKNG (1, 154)
BKR (1, 154)
BMRN (1, 154)
BNTX (1, 154)
BSY (1, 154)
CCEP (1, 154)
CDNS (1, 156)
CDW (1, 154)
CEG (1, 153)
CG (1, 155)
CHK (1, 153)
CHKP (1, 153)
CHRW (1, 154)
CHTR (1, 154)
CINF (1, 154)
CMCSA (1, 155)
CME (1, 154)
COST (1, 154)
CPRT (1, 156)
CRWD (1, 154)
CSCO (1, 154)
CSGP (1, 154)
CSX (1, 155)
CTAS (1, 155)
CTSH (1, 154)
DDOG (1, 154)
DLTR (1, 154)
DOCU (1, 154)
DOX (1, 156)
DXCM (1, 154)
EA (1, 154)
EBAY (1, 154)
ENPH (1, 154)
EQIX (1, 154)
ERIC (1, 153)
ERIE (1, 155)
ETSY (1, 154)
EXC (1, 155)
EXPD (1, 154)
EXPE (1, 153)
FANG (1, 154)
FAST (1, 154)
FCNCA (1, 154)
FISV (1, 154)
FITB (1, 

In [ ]:
wkinfo.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkinfo, dataframe=info, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
rec = Recommendations(market)    #Get all recommendations data for all exchanges

In [ ]:
wkrec.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkrec, dataframe=rec, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
fin = Financials(market)      #Get all financial data for all exchanges

Df 1 shape:  (4, 61)
Df 2 shape:  (0, 1)
Df 3 shape:  (4, 38)
2035
KB
Df 1 shape:  (0, 1)
Df 2 shape:  (0, 1)
Df 3 shape:  (0, 1)
2036
KEP
Df 1 shape:  (0, 1)
Df 2 shape:  (4, 107)
Df 3 shape:  (4, 67)
2041
KEY
Df 1 shape:  (4, 57)
Df 2 shape:  (0, 1)
Df 3 shape:  (4, 46)
2046
KEYS
Df 1 shape:  (0, 1)
Df 2 shape:  (4, 71)
Df 3 shape:  (4, 44)
2051
KIM
Df 1 shape:  (0, 1)
Df 2 shape:  (4, 57)
Df 3 shape:  (4, 57)
2056
KKR
Df 1 shape:  (0, 1)
Df 2 shape:  (4, 74)
Df 3 shape:  (4, 52)
2061
KMB
Df 1 shape:  (4, 58)
Df 2 shape:  (0, 1)
Df 3 shape:  (0, 1)
2066
KMI
Df 1 shape:  (0, 1)
Df 2 shape:  (4, 69)
Df 3 shape:  (0, 1)
2071
KO
Df 1 shape:  (0, 1)
Df 2 shape:  (0, 1)
Df 3 shape:  (0, 1)
2072
KOF
Df 1 shape:  (4, 70)
Df 2 shape:  (0, 1)
Df 3 shape:  (4, 57)
2077
KR
Df 1 shape:  (4, 63)
Df 2 shape:  (0, 1)
Df 3 shape:  (4, 50)
2082
L
Df 1 shape:  (4, 48)
Df 2 shape:  (0, 1)
Df 3 shape:  (4, 33)
2087
LDOS
Df 1 shape:  (4, 55)
Df 2 shape:  (0, 1)
Df 3 shape:  (0, 1)
2092
LEN
Df 1 shape:  (3

In [ ]:
fin

,Symbol,Date,Operating Cashflow,Free Cashflow,Current Liabilities,Working Capital,Invested Capital,Total Revenue,Cost of Revenue,Gross Profit,EBIT,Operating Cashflow Ratio
0,AAPL,2022-09-30,122151000000.0,111443000000.0,153982000000.0,-18577000000.0,170741000000.0,394328000000.0,223546000000.0,170782000000.0,122034000000.0,0.793281
1,AAPL,2021-09-30,104038000000.0,92953000000.0,125481000000.0,9355000000.0,187809000000.0,365817000000.0,212981000000.0,152836000000.0,111852000000.0,0.829114
2,AAPL,2020-09-30,80674000000.0,73365000000.0,105392000000.0,38321000000.0,177775000000.0,274515000000.0,169559000000.0,104956000000.0,69964000000.0,0.765466
3,AAPL,2019-09-30,69391000000.0,58896000000.0,105718000000.0,57101000000.0,198535000000.0,260174000000.0,161782000000.0,98392000000.0,69313000000.0,0.656378
0,ABMD,2022-03-31,285390000.0,249627000.0,138457000.0,838016000.0,1503326000.0,1031753000.0,188158000.0,843595000.0,256706000.0,2.061218
...,...,...,...,...,...,...,...,...,...,...,...,...
0,LNG,2021-12-31,2469000000.0,1503000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LNG,2020-12-31,1265000000.0,-574000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LNG,2019-12-31,1833000000.0,-1223000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LNG,2018-12-31,1990000000.0,-1653000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
fin2 = fin.dropna(how = "all")

In [ ]:
fin2

,Symbol,Date,Operating Cashflow,Free Cashflow,Current Liabilities,Working Capital,Invested Capital,Total Revenue,Cost of Revenue,Gross Profit,EBIT,Operating Cashflow Ratio
0,AAPL,2022-09-30,122151000000.0,111443000000.0,153982000000.0,-18577000000.0,170741000000.0,394328000000.0,223546000000.0,170782000000.0,122034000000.0,0.793281
1,AAPL,2021-09-30,104038000000.0,92953000000.0,125481000000.0,9355000000.0,187809000000.0,365817000000.0,212981000000.0,152836000000.0,111852000000.0,0.829114
2,AAPL,2020-09-30,80674000000.0,73365000000.0,105392000000.0,38321000000.0,177775000000.0,274515000000.0,169559000000.0,104956000000.0,69964000000.0,0.765466
3,AAPL,2019-09-30,69391000000.0,58896000000.0,105718000000.0,57101000000.0,198535000000.0,260174000000.0,161782000000.0,98392000000.0,69313000000.0,0.656378
0,ABMD,2022-03-31,285390000.0,249627000.0,138457000.0,838016000.0,1503326000.0,1031753000.0,188158000.0,843595000.0,256706000.0,2.061218
...,...,...,...,...,...,...,...,...,...,...,...,...
4,IMO,2018-12-31,NaN,NaN,NaN,NaN,NaN,33297000000.0,29217000000.0,4080000000.0,3181000000.0,NaN
0,LNG,2021-12-31,2469000000.0,1503000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LNG,2020-12-31,1265000000.0,-574000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LNG,2019-12-31,1833000000.0,-1223000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
fin = fin2

In [ ]:
fin2 = fin2.dropna(thresh = 4)

In [ ]:
import numpy as np
fin2 = fin2.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
wkfin.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkfin, dataframe=fin, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
share = Shareholders(market)    #Get all shareholder data for all exchanges

AAPL 0.0
ABMD 0.1364256480218281
ABNB 0.2728512960436562
ACGL 0.4092769440654843
ADBE 0.5457025920873124
ADI 0.6821282401091405
ADP 0.8185538881309686
ADSK 0.9549795361527967
AEP 1.0914051841746248
AKAM 1.227830832196453
ALGN 1.364256480218281
ALNY 1.5006821282401093
AMAT 1.6371077762619373
AMD 1.7735334242837655
AMGN 1.9099590723055935
AMZN 2.046384720327422
ANSS 2.1828103683492497
APA 2.3192360163710775
ARGX 2.455661664392906
ASML 2.592087312414734
ATVI 2.728512960436562
AVGO 2.8649386084583903
AXON 3.0013642564802185
AZN 3.1377899045020468
AZPN 3.2742155525238745
BGNE 3.4106412005457027
BIDU 3.547066848567531
BIIB 3.6834924965893587
BKNG 3.819918144611187
BKR 3.956343792633015
BMRN 4.092769440654844
BNTX 4.229195088676671
BSY 4.365620736698499
CCEP 4.502046384720328
CDNS 4.638472032742155
CDW 4.774897680763983
CEG 4.911323328785812
CG 5.0477489768076405
CHK 5.184174624829468
CHKP 5.320600272851296
CHRW 5.457025920873124
CHTR 5.5934515688949515
CINF 5.729877216916781
CMCSA 5.86630286

In [ ]:
wkshare.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkshare, dataframe=share, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
sus = Sustainability(market)    #Get all sustainability data for all exchanges

AAPL 0.0
ABMD 0.1364256480218281
ABNB 0.2728512960436562
ACGL 0.4092769440654843
ADBE 0.5457025920873124
ADI 0.6821282401091405
ADP 0.8185538881309686
ADSK 0.9549795361527967
AEP 1.0914051841746248
AKAM 1.227830832196453
ALGN 1.364256480218281
ALNY 1.5006821282401093
AMAT 1.6371077762619373
AMD 1.7735334242837655
AMGN 1.9099590723055935
AMZN 2.046384720327422
ANSS 2.1828103683492497
APA 2.3192360163710775
ARGX 2.455661664392906
ASML 2.592087312414734
ATVI 2.728512960436562
AVGO 2.8649386084583903
AXON 3.0013642564802185
AZN 3.1377899045020468
AZPN 3.2742155525238745
BGNE 3.4106412005457027
BIDU 3.547066848567531
BIIB 3.6834924965893587
BKNG 3.819918144611187
BKR 3.956343792633015
BMRN 4.092769440654844
BNTX 4.229195088676671
BSY 4.365620736698499
CCEP 4.502046384720328
CDNS 4.638472032742155
CDW 4.774897680763983
CEG 4.911323328785812
CG 5.0477489768076405
CHK 5.184174624829468
CHKP 5.320600272851296
CHRW 5.457025920873124
CHTR 5.5934515688949515
CINF 5.729877216916781
CMCSA 5.86630286

In [ ]:
wksus.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wksus, dataframe=sus, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
news = News(market)     #Get all news data for all exchanges

AAPL 0.0
ABMD 0.1364256480218281
ABNB 0.2728512960436562
ACGL 0.4092769440654843
ADBE 0.5457025920873124
ADI 0.6821282401091405
ADP 0.8185538881309686
ADSK 0.9549795361527967
AEP 1.0914051841746248
AKAM 1.227830832196453
ALGN 1.364256480218281
ALNY 1.5006821282401093
AMAT 1.6371077762619373
AMD 1.7735334242837655
AMGN 1.9099590723055935
AMZN 2.046384720327422
ANSS 2.1828103683492497
APA 2.3192360163710775
ARGX 2.455661664392906
ASML 2.592087312414734
ATVI 2.728512960436562
AVGO 2.8649386084583903
AXON 3.0013642564802185
AZN 3.1377899045020468
AZPN 3.2742155525238745
BGNE 3.4106412005457027
BIDU 3.547066848567531
BIIB 3.6834924965893587
BKNG 3.819918144611187
BKR 3.956343792633015
BMRN 4.092769440654844
BNTX 4.229195088676671
BSY 4.365620736698499
CCEP 4.502046384720328
CDNS 4.638472032742155
CDW 4.774897680763983
CEG 4.911323328785812
CG 5.0477489768076405
CHK 5.184174624829468
CHKP 5.320600272851296
CHRW 5.457025920873124
CHTR 5.5934515688949515
CINF 5.729877216916781
CMCSA 5.86630286

In [ ]:
wknews.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wknews, dataframe=news, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
pt = PriceTarget(market)    #Get all price target data for all exchanges

yfinance.Ticker object <AAPL>
yfinance.Ticker object <ABMD>
yfinance.Ticker object <ABNB>
yfinance.Ticker object <ACGL>
yfinance.Ticker object <ADBE>
yfinance.Ticker object <ADI>
yfinance.Ticker object <ADP>
yfinance.Ticker object <ADSK>
yfinance.Ticker object <AEP>
yfinance.Ticker object <AKAM>
yfinance.Ticker object <ALGN>
yfinance.Ticker object <ALNY>
yfinance.Ticker object <AMAT>
yfinance.Ticker object <AMD>
yfinance.Ticker object <AMGN>
yfinance.Ticker object <AMZN>
yfinance.Ticker object <ANSS>
yfinance.Ticker object <APA>
yfinance.Ticker object <ARGX>
yfinance.Ticker object <ASML>
yfinance.Ticker object <ATVI>
yfinance.Ticker object <AVGO>
yfinance.Ticker object <AXON>
yfinance.Ticker object <AZN>
yfinance.Ticker object <AZPN>
yfinance.Ticker object <BGNE>
yfinance.Ticker object <BIDU>
yfinance.Ticker object <BIIB>
yfinance.Ticker object <BKNG>
yfinance.Ticker object <BKR>
yfinance.Ticker object <BMRN>
yfinance.Ticker object <BNTX>
yfinance.Ticker object <BSY>
yfinance.Ticker ob

In [ ]:
wkpt.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkpt, dataframe=pt, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
fore = Forecasts(market)    #Get all forecast data for all exchanges

AAPL 0.0 <class 'pandas.core.frame.DataFrame'> (6, 8)
ABMD 0.1375515818431912 <class 'pandas.core.frame.DataFrame'> (6, 8)
ABNB 0.2751031636863824 <class 'pandas.core.frame.DataFrame'> (6, 8)
ACGL 0.41265474552957354 <class 'pandas.core.frame.DataFrame'> (6, 8)
ADBE 0.5502063273727648 <class 'pandas.core.frame.DataFrame'> (6, 8)
ADI 0.6877579092159559 <class 'pandas.core.frame.DataFrame'> (6, 8)
ADP 0.8253094910591471 <class 'pandas.core.frame.DataFrame'> (6, 8)
ADSK 0.9628610729023385 <class 'pandas.core.frame.DataFrame'> (6, 8)
AEP 1.1004126547455295 <class 'pandas.core.frame.DataFrame'> (6, 8)
AKAM 1.2379642365887207 <class 'pandas.core.frame.DataFrame'> (6, 8)
ALGN 1.3755158184319118 <class 'pandas.core.frame.DataFrame'> (6, 8)
ALNY 1.5130674002751032 <class 'pandas.core.frame.DataFrame'> (6, 8)
AMAT 1.6506189821182942 <class 'pandas.core.frame.DataFrame'> (6, 8)
AMD 1.7881705639614855 <class 'pandas.core.frame.DataFrame'> (6, 8)
AMGN 1.925722145804677 <class 'pandas.core.frame.Dat

In [ ]:
wkfore.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkfore, dataframe=fore, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

## Further Data Cleaning

In [ ]:
def PriceT(PT):   #Function to determine null value symbols in Price target
  df = pd.DataFrame(columns = ["Symbol"])   #format output dataframe
  df1 = PT[PT["Target Low Price"].isna()]   #Check for null values
  df2 = PT[PT["Target Mean Price"].isna()]
  df3 = PT[PT["Target High Price"].isna()]
  df4 = PT[PT["Number of Analyst Opinions"].isna()]
  print(len(df1),len(df2),len(df3),len(df4))
  df6 = pd.concat([df1,df2,df3,df4])    #combine all missing values
  df5 = df6.drop_duplicates()           #drop any duplicates
  df["Symbol"] = df5["Symbol"]          #store only the symbol column
  return df   #return the output

def EG(PT):   #Function to determine null value symbols in Forecasts (same process as above)
  df = pd.DataFrame(columns = ["Symbol"])
  df1 = PT[PT["Earnings Growth"].isna()]
  df2 = PT[PT["Sales Growth"].isna()]
  df12 = pd.concat([df1,df2])
  df3 = df12.drop_duplicates()
  df["Symbol"] = df3["Symbol"]
  return df

def CInfo(PT):    #Function to determine null value symbols in Info (same process as above)
  df = pd.DataFrame(columns = ["Symbol"])
  df1 = PT[PT["Name"].isna()]
  df2 = PT[PT["Country"].isna()]
  df3 = PT[PT["Industry"].isna()]
  df4 = PT[PT["Sector"].isna()]
  df5 = PT[PT["Summary"].isna()]
  df6 = PT[PT["Logo URL"].isna()]
  df7 = PT[PT["Employees"].isna()]
  df8 = PT[PT["Profit Margins"].isna()]
  df9 = PT[PT["Revenue Growth"].isna()]
  print(len(df1),len(df2),len(df3),len(df4),len(df5),len(df6),len(df7),len(df8),len(df9),)
  df12 = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9])
  df23 = df12.drop_duplicates()
  df["Symbol"] = df23["Symbol"]
  return df

def Remove(Re, c):    #function to remove rows containing the symbols contained in dataframe c from the stock data
  for i in range(len(c)):   #iterate through dataframe c
    a = c.iloc[i,0]         #get the symbol value
    Re = Re[Re["Symbol"]!= a]   #only keep rows that don not contain the symbol stored in a
  return Re   #return the reduced dataframe

def Dup(check): #function to check for duplicate names in the info dataframe
  c = check.groupby(["Name"]).cumcount()  
  c = c.replace(0, '').astype(str)
  check["Name"] += c
  check["Name"] = check["Name"].str.replace("1", "#") #add "#" at the end of duplicate names to distinguish them
  return check    #return the duplicate free dataframe

def Different(pt, info, fore):   #function to find symbols that are not common in all outputs
  d1 = pd.DataFrame(pt["Symbol"], columns = ["Symbol"])
  d2 = pd.DataFrame(info["Symbol"], columns = ["Symbol"])
  d3 = pd.DataFrame(fore["Symbol"], columns = ["Symbol"])
  dif1 = Remove(d1, d2)
  dif12 = Remove(d2,d1)
  dif2 = Remove(d1, d3)
  dif22 = Remove(d3,d1)
  dif3 = Remove(d2,d3)
  dif32 = Remove(d3,d2)
  dif = pd.concat([dif1,dif12,dif2,dif22,dif3,dif32])
  dif = dif.drop_duplicates()
  return dif

In [ ]:
df1 = PriceT(pt)
print(df1)

36 36 36 36
  Symbol
0   ACGL
0    DOX
0   ERIE
0    IEP
0   NTES
0    STX
0   TCOM
0    WTW
0    AER
0     BA
0   BBVA
0     BG
0    CAT
0     CB
0    CHT
0    DIS
0   GRMN
0    JCI
0    KEP
0      L
0    LHX
0    LLY
0     MS
0   MUFG
0    NGG
0    PFE
0    PKX
0     RE
0    RTO
0    SJR
0    SLB
0    SUZ
0    TEL
0     TT
0    TXT
0    WDS


In [ ]:
df2 = EG(fore)
print(df2)

Empty DataFrame
Columns: [Symbol]
Index: []


In [ ]:
df3 = CInfo(info)
print(df3)

0 0 0 0 0 0 0 0 24
  Symbol
0  CMCSA
0   RIVN
0    WDC
0    WTW
0    XEL
0    AMT
0    DTE
0    KKR
0     MA
0    MAA
0    MCD
0    MFC
0    MMM
0   MSCI
0    NLY
0    PCG
0    PUK
0     RS
0    SAN
0    SWK
0   TECK
0    TFX
0    TXT
0    WST


In [ ]:
df4 = pd.concat([df1,df2,df3])
df5 = df4.drop_duplicates()

In [ ]:
info = Dup(info)

In [ ]:
pt = Remove(pt, df5)

In [ ]:
info = Remove(info, df5)

In [ ]:
fore = Remove(fore,df5)

In [ ]:
pt["Symbol"].nunique()

669

In [ ]:
info["Symbol"].nunique()

669

In [ ]:
fore["Symbol"].nunique()

637

In [ ]:
dif = Different(pt, info, fore)

In [ ]:
len(dif)

32

In [ ]:
pt = Remove(pt, dif)

In [ ]:
pt["Symbol"].nunique()    #compare number of unique symbols to ensure all data sets are using the same symbols

637

In [ ]:
wkpt.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkpt, dataframe=pt, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
info = Remove(info, dif)

In [ ]:
info["Symbol"].nunique()

637

In [ ]:
wkinfo.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkinfo, dataframe=info, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
fore = Remove(fore, dif)

In [ ]:
fore["Symbol"].nunique()

637

In [ ]:
wkfore.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkfore, dataframe=fore, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
n = Remove(n, df5)

In [ ]:
n["Symbol"].nunique() 

201

In [ ]:
n = Remove(n, dif)

In [ ]:
n["Symbol"].nunique()   #sum of the three exchanges symbols should equal the unique values of the others

199

In [ ]:
wknasdaq.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wknasdaq, dataframe=n, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
ny = Remove(ny, df5)

In [ ]:
ny["Symbol"].nunique() 

465

In [ ]:
ny = Remove(ny, dif)

In [ ]:
ny["Symbol"].nunique()

435

In [ ]:
wknyse.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wknyse, dataframe=ny, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet

In [ ]:
a = Remove(a, df5)

In [ ]:
a["Symbol"].nunique() 

3

In [ ]:
a = Remove(a, dif)

In [ ]:
a["Symbol"].nunique()

3

In [ ]:
wkamex.clear()      #Clear the Sheet
set_with_dataframe(worksheet=wkamex, dataframe=a, include_index=False,
include_column_header=True, resize=True)      #put DataFrame into sheet